In [144]:
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
import numpy as np
from sklearn.model_selection import train_test_split
from detect_outlier import *


In [145]:
df = read_data()


In [146]:
df.loc[df['sub_label'].isna(), 'sub_label'] = df['main_label']
df = df.dropna()



In [147]:
df_normalized = df.drop(["address", "createdAt"], axis=1)
main_label_column = df['main_label']
sub_label_column = df['sub_label']

for column in df_normalized.columns:
    if column not in ["numberOfLiquidation", "borrow_per_deposit"]:
        min_val = df_normalized[column].min()
        max_val = df_normalized[column].max()
        df_normalized[column] = ((df_normalized[column] - min_val) /
                                         (max_val - min_val)) * 550 + 300

min_max_columns = ["numberOfLiquidation", "borrow_per_deposit"]

for column in min_max_columns:
    min_val = df_normalized[column].min()
    max_val = df_normalized[column].max()
    df_normalized[column] = ((max_val - df_normalized[column]) /
                                     (max_val - min_val)) * 550 + 300

df_normalized['main_label'] = main_label_column
df_normalized['sub_label'] = sub_label_column


In [148]:
# df_normalized =df.drop("address", axis= 1).drop("createdAt", axis= 1).drop("zscore", axis= 1)
# label_column = df['label']
# df_normalized = (df_normalized - df_normalized.min()) / (df_normalized.max() - df_normalized.min())
# df_normalized = df_normalized * 550 + 300
# df_normalized['label'] = label_column


In [149]:
df_normalized.head()


,balanceInUSD,totalValueOfLiquidation,numberOfLiquidation,frequencyOfDappTransactions,numberOfInteractedDapps,typesOfInteractedDapps,numberOfReputableDapps,averageTotalAsset,frequencyOfTransaction,frequencyMountOfTransaction,main_label,sub_label,borrow_per_balance,deposit_per_balance,borrow_per_deposit,age
2,300.0,300.0,850.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,1.0,1.0,300.0,300.0,850.0,351.962578
3,300.0,300.0,850.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,1.0,1.0,300.0,300.0,850.0,472.437869
4,300.0,300.0,850.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,1.0,1.0,300.0,300.0,850.0,470.458973
6,300.0,300.0,850.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,1.0,1.0,300.0,300.0,850.0,606.350564
8,300.0,300.0,850.0,300.0,300.0,300.0,300.0,300.0,300.0,300.0,1.0,1.0,300.0,300.0,850.0,462.272912


In [150]:
X = np.array(df_normalized[
    [
        "balanceInUSD",
        # "borrowInUSD",
        # "depositInUSD",
        "averageTotalAsset",
        "frequencyMountOfTransaction",
        "borrow_per_balance",
        "deposit_per_balance",
        "borrow_per_deposit",
        "totalValueOfLiquidation",
        "numberOfLiquidation",
        "frequencyOfTransaction",
        "frequencyOfDappTransactions",
        "numberOfInteractedDapps",
        "typesOfInteractedDapps",
        "numberOfReputableDapps",
        "age",
    ]
].values)
main_y = df_normalized["main_label"].values
sub_y = df_normalized["sub_label"].values
# y = np.array([score(y[i]) for i in range(len(y))])



In [151]:
import numpy as np
def nomal(arr):
    sum = np.sum(arr)
    arr = arr / sum
    return arr

In [152]:
import random
import numpy as np
from sklearn.model_selection import train_test_split

class Genetic_Alogorithm:
    def __init__(
        self, X, main_y, sub_y, num_params, num_solutions=100, mutation_rate=0.2
    ):
        self.X = X
        self.main_y = main_y
        self.sub_y = sub_y
        self.num_params = num_params
        self.num_solutions = num_solutions
        self.mutation_rate = mutation_rate
        (
            self.X_train,
            self.X_test,
            self.y_train,
            self.y_test,
            self.sub_y_train,
            self.sub_y_test,
        ) = train_test_split(
            X, main_y, sub_y, test_size=0.2, stratify=main_y, random_state=42
        )

    def initialize_solutions(self):
        solutions = []
        for _ in range(self.num_solutions):
            # Sử dụng np.random.uniform để khởi tạo giá trị trong khoảng từ -500 đến 500
            solution = np.random.uniform(0, 1, size=self.num_params)
            solutions.append(tuple(nomal(solution)))
        return solutions

    def mutate(self, child, mutation_rate):
        if mutation_rate is None:
            mutation_rate = self.mutation_rate
        mutated_child = tuple(
            [
                gene * np.random.uniform(1 - self.mutation_rate, 1 + self.mutation_rate)
                for gene in child
            ]
        )
        return mutated_child

    def crossover(self, parent1, parent2, type=None):
        if type is None or type == "one_point":
            crossover_point = np.random.randint(1, len(parent1) - 1)
            child = list(parent1[:crossover_point]) + list(parent2[crossover_point:])
            return tuple(child)
        elif type == "two_point":
            crossover_point1, crossover_point2 = np.sort(
                np.random.choice(range(1, len(parent1)), 2, replace=False)
            )
            child = (
                list(parent1[:crossover_point1])
                + list(parent2[crossover_point1:crossover_point2])
                + list(parent1[crossover_point2:])
            )
            return tuple(nomal(child))
        elif type == "mean":
            child = [(parent1[i] + parent2[i]) / 2 for i in range(len(parent1))]
            return tuple(child)

    def evolve(self, solutions, crossover="None", mutate_rate="None"):
        rankedsolutions = [(self.fitness(theta), theta) for theta in solutions]
        rankedsolutions = sorted(rankedsolutions, key=lambda x: x[0], reverse=True)
        print(f"fitness:{rankedsolutions[0][0]}")
        bestSolutions = rankedsolutions[:20] + rankedsolutions[-5:]

        new_solution = [rankedsolutions[0][1]]

        for _ in range(self.num_solutions - 1):
            parent1, parent2 = (
                random.choice(bestSolutions)[1],
                random.choice(bestSolutions)[1],
            )
            child1 = self.crossover(parent1, parent2, crossover)
            child1 = self.mutate(child1, mutate_rate)
            new_solution.append(nomal(child1))
        return new_solution, rankedsolutions[0][0]

    # support funcion
    def fitness(self, theta):
        result = 0
        y_pred = self.predict(self.X_train, theta)
        return self.accuracy_score(self.y_train, self.sub_y_train, y_pred)
        for yp, yt in zip(y_pred, self.y_train):
            diff = abs(yt - yp)
            if diff == 0:
                continue
            elif diff == 1:
                result += 200
            elif diff == 2:
                result += 400
            elif diff == 3:
                result += 800
            else:
                result += 2000
        return result

    def accuracy_score(self, y_train, sub_y_train, y_pred):
        y_train = np.array(y_train)
        sub_y_train = np.array(sub_y_train)
        y_pred = np.array(y_pred)
        condition = np.logical_or(y_pred == sub_y_train, y_pred == y_train)
        count = np.sum(condition)
        accuracy = count / len(y_train)
        return accuracy

    def predict(self, matrices, theta):
        list_scores = np.dot(matrices, theta)
        label = []
        for score in list_scores:
            if  score < 580:
                label.append(0)
            elif score >= 580 and score < 670:
                label.append(1)
            elif score >= 670 and score < 740:
                label.append(2)
            elif score >= 740 and score < 800:
                label.append(3)
            elif 800<= score <= 850:
                label.append(4)
            else:
                print(score)
        return np.array(label)

In [153]:
ga = Genetic_Alogorithm(X, main_y, sub_y, len(X[0]))

In [154]:
solution = ga.initialize_solutions()
fitness = []
num_generations = 100
for _ in range(num_generations):
    solution, acc = ga.evolve(solution, crossover="two_point", mutate_rate=0.1)
    fitness.append(acc)

fitness:0.010563091676570633
fitness:0.010563091676570633
fitness:0.010563091676570633
fitness:0.010563091676570633
fitness:0.010563091676570633
fitness:0.010573337158313184
fitness:0.010573337158313184
fitness:0.012981025367812794
fitness:0.11363263800663907
fitness:0.8405700586041556
fitness:0.8410720872095406
fitness:0.8410720872095406
fitness:0.8411233146182534
fitness:0.8411233146182534
fitness:0.8411233146182534
fitness:0.8411233146182534
fitness:0.8411335600999958
fitness:0.8411335600999958
fitness:0.8411642965452235
fitness:0.8411642965452235
fitness:0.8411745420269661
fitness:0.8411745420269661
fitness:0.8411745420269661
fitness:0.8411745420269661
fitness:0.8411847875087086
fitness:0.8411847875087086
fitness:0.8411847875087086
fitness:0.8411847875087086
fitness:0.8411847875087086
fitness:0.8411847875087086
fitness:0.8411847875087086
fitness:0.8411950329904512
fitness:0.8411950329904512
fitness:0.8411950329904512
fitness:0.8411950329904512
fitness:0.8411950329904512
fitness:0.8

In [155]:
solution[0]

array([0.00358738, 0.1572907 , 0.00563742, 0.01807542, 0.01033427,
       0.0298886 , 0.00609766, 0.47987563, 0.00062043, 0.1063417 ,
       0.09303763, 0.05459331, 0.03082876, 0.00379109])